# Simple Fully Connected Autoencoder

In [1]:
from river.compose import Pipeline
from river.preprocessing import MinMaxScaler

from river_torch.anomaly import Autoencoder, ProbabilityWeightedAutoencoder
from river import metrics
from river.datasets import CreditCard
from torch import nn, manual_seed
import math

In [7]:
import numpy as np
import collections

observed_classes = []


def add_list():
    y = np.random.randint(0, 100000, size=10000)
    for y_i in y:
        if y_i not in observed_classes:
            observed_classes.append(y_i)

def add_counter():
    y = np.random.randint(0, 100000, size=10000)
    counter = collections.Counter()
    for y_i in y:
        counter.update(y)


In [4]:
%timeit add_list()

5.21 s ± 1.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit add_counter()

14.9 s ± 1.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
_ = manual_seed(42)
dataset = CreditCard().take(5000)
metric = metrics.ROCAUC(n_thresholds=50)


class MyAutoEncoder(nn.Module):
    def __init__(self, n_features, latent_dim=3):
        super(MyAutoEncoder, self).__init__()
        self.linear1 = nn.Linear(n_features, latent_dim)
        self.nonlin = nn.LeakyReLU()
        self.linear2 = nn.Linear(latent_dim, n_features)

    def forward(self, X, **kwargs):
        X = self.linear1(X)
        X = self.nonlin(X)
        X = self.linear2(X)
        return nn.functional.sigmoid(X)


ae = Autoencoder(module=MyAutoEncoder, lr=0.005)
scaler = MinMaxScaler()
model = Pipeline(scaler, ae)

for x, y in dataset:
    score = model.score_one(x)
    metric.update(y_true=y, y_pred=score)
    model.learn_one(x=x)
print(f"ROCAUC: {metric.get():.4f}")


ROCAUC: 0.6292
